Instructions: Run (press play) on all boxes once and then play with the controls that appear in the bottom (such as the sliders).


Coherency
Understanding coherency in an intuitive manner with modifyiable examples. Coherency being a complex measure that tells us information about the mean relationship of two sources (such as two electrodes) regarding their phase lag and amplitude.
First we construct the sources  x(t)x(t)  and  y(t)y(t)  in the time domain as a linear mixture of three signals,  s1(t),s2(t)s1(t),s2(t) , and  s3(t)s3(t) . We also take a phase-shifted version of  s2(t)s2(t) .

In [1]:
# (*) To communicate with Plotly's server, sign in with credentials file
import plotly.plotly as py
# We plot both cross-spectra
import plotly.graph_objs as go
# (*) Useful Python/Plotly tools
from plotly.graph_objs import Line  # (*) import Line 

import plotly.tools as tls

import numpy as np  # (*) numpy for math functions and arrays

import json

import plotly

from ipywidgets import interactive

from plotly.graph_objs import *

from notebook import *

from IPython.display import display, clear_output

from scipy.fftpack import fft, fftn
from scipy.signal import hanning

from plotly.widgets import GraphWidget


/Users/andrea/anaconda3/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning:

The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.

/Users/andrea/anaconda3/lib/python3.5/site-packages/IPython/utils/traitlets.py:5: UserWarning:

IPython.utils.traitlets has moved to a top-level traitlets package.



<IPython.core.display.Javascript object>

We chose a "recording" time of 6 seconds, sampling at 1000 Hz. Further on in the analysis we divide the 6 seconds recording period into epochs of length  TT . We chose  T=500msT=500ms . There are a total of 1000 trials.


In [2]:
time = 6 # in seconds: time of recording per trial
Fs = 1000 # in Hertz: sampling frequency
T = 500 # in ms: time of an epoch

# We can already calculate the interrelationships between time and frequency domains
freq_res = 1/(T/1000) # frequency resolution
N = Fs*(T/1000) # number of samples in an epoch
Fmax = (N/2)*freq_res # maximal frequency or nyquist frequency
W = np.linspace(0,Fmax,T)*freq_res # here we store the frequency indices

# For the statistics
num_trials = 1000


The relationship between the signals and the sources is fixed. The sources are computed according to the following rule:
x(t)=s1(t)+1.5s2(t)+s3(t)+ϵ
x(t)=s1(t)+1.5s2(t)+s3(t)+ϵ
 
y(t)=0.5s1(t)+α(t)s2s(t)+β(t)s2(t)+ϵ
y(t)=0.5s1(t)+α(t)s2s(t)+β(t)s2(t)+ϵ
 
 

Where  s2s(t)s2s(t)  is the shifted version of  s2(t)s2(t) , and  ϵϵ  is white noise.
The coefficients  αα  and  ββ  are step functions that are illustrated later

In [3]:
def sources(phase_val, noise_val, freq1_val, freq2_val, freq3_val, num_trials=1):
    noise_coef = noise_val # noise coefficient
    t = np.linspace(0,time*Fs,time*Fs,endpoint=True) # time points in miliseconds, 10 seconds at 1000 Hz
    freq1 = freq1_val # Hz, first frequency
    freq2 = freq2_val # Hz, second frequency 
    freq3 = freq3_val # Hz, third frequency
    
    phase_shift = phase_val*2*np.pi # in radians
    
    s1t = np.ones((num_trials,time*Fs))*np.sin(2*np.pi*t*freq1/Fs) # signal 1
    s2_1t = np.ones((num_trials,time*Fs))*np.sin(2*np.pi*(t*freq2/Fs)) # signal 2
    s2_2t = np.ones((num_trials,time*Fs))*np.sin(2*np.pi*(t*freq2/Fs)+ phase_shift) # signal 2 with phase shift
    s3t = np.ones((num_trials,time*Fs))*np.sin(2*np.pi*t*freq3/Fs) # signal 3
    
    alpha = np.ones((num_trials,time*Fs))*np.linspace(-1,1,time*Fs,endpoint=True)
    alpha = np.sign(alpha) +1
    beta = np.sign(-alpha) +1
    Xt = s1t + 1.5*s2_1t + s3t + noise_coef*np.random.randn(num_trials,time*Fs)
    Yt = 0.5*s1t + alpha*s2_2t + beta*s2_1t + noise_coef*np.random.randn(num_trials,time*Fs)
    return t,Xt,Yt,s1t,s2_1t,s2_2t,s3t,alpha,beta

In [4]:
def coherency(time,f,Xt,Yt,Fs=1000,T=500):
    C_xy_f_t = ()
    trials = np.shape(Xt)[0]
    H_window = np.ones((trials,T))*hanning(T)
    # First we convert f to columns according to the resolution of our fft
    w = np.int(f/freq_res)
    
    for segs in range(np.int(np.floor(time*Fs/T))):
        #Making a matrix of the segments of Xt and Yt during a specific time through all trials
        X = Xt[:,segs*T:(segs+1)*T]
        Y = Yt[:,segs*T:(segs+1)*T]

        Xf = np.zeros((time*Fs/T,T),dtype="complex_")
        Yf = np.zeros((time*Fs/T,T),dtype="complex_")

        Xf=fftn(X*H_window)
        Yf=fftn(Y*H_window)
        
        # We approximate the expectation value with the mean over all the epochs
        # We will take a narrow band to include +-2 Hz around f
        S_xy_f_t = np.mean(Xf[:,w-1:w+1]*np.conj(Yf[:,w-1:w+1]))
        S_xx_f_t = np.mean(Xf[:,w-1:w+1]*np.conj(Xf[:,w-1:w+1]))
        S_yy_f_t = np.mean(Yf[:,w-1:w+1]*np.conj(Yf[:,w-1:w+1]))

        # We now construct the denominator of the coherency
        deno_f_t = np.sqrt(np.real(S_xx_f_t*S_yy_f_t))
        
        # We finally get the coherency
        C_xy_f_t = np.append(C_xy_f_t,S_xy_f_t/deno_f_t)
        #print(deno_f_t,C_xy_f_t)
        
    return C_xy_f_t

Step 1. Construct your  signalssignals .
Step 2. Look at your  sourcessources  and choose the  noisenoise  level.
Step 3. Choose the  frequencyfrequency  at which to compute the coherency.
Step 4. See what the  coherencycoherency  looks like.
Step 5. Modify any of the parameters and see how things change. Have fun!

In [5]:
def regraph(slider_phase,slider_noise,slider_freq1,slider_freq2,slider_freq3,slider_f):
    tt,Xt,Yt,s1t,s2_1t,s2_2t,s3t,alpha,beta = sources(slider_phase/100, slider_noise/100,slider_freq1,slider_freq2,slider_freq3,num_trials)
    xt = Xt[0,:]
    yt = Yt[0,:]
    cohe = coherency(time,slider_f,Xt,Yt)
    Cxy_f = np.mean(cohe)
    
    # Since all coherency values lay on the unit circle, we actually only care about their angle. 
    # To extract the angle we use the arctan2() function. In this way we go from a complex number 
    # to a real one. The arctan2() function returns the “four quadrant” arctan of the angle formed 
    # by (x, y) and the positive x-axis. It returns values in radians in the (-pi, pi) range.

    angles = np.arctan2(np.imag(cohe),np.real(cohe))
    bb = 80
    counts,bins = np.histogram(angles+np.pi, bins=bb, range=[0,2*np.pi*(1 + 1/bb)], normed=True, weights=None, density=None)
    
    g1.restyle({'x': [[]], 'y': [[]]})
    g1.add_traces(dict(x=tt, y=list(3+s1t[0,]), name = 'Signal 1: s1(t)'))                     
    g1.add_traces(dict(x=tt, y=list(s3t[0,]), name = 'Signal 3: s3(t)'))                     
    g1.add_traces(dict(x=tt, y=list(-3+s2_1t[0,]), name = 'Signal 2: s2(t)'))
    g1.add_traces(dict(x=tt, y=list(-6+s2_2t[0,]), name = 'Signal 2 with phase shift: s2_shift(t)'))
    g1.add_traces(dict(x=tt, y=list(-3+alpha[0,]), name = 'Coefficient for s2(t)'))
    g1.add_traces(dict(x=tt, y=list(-6+beta[0,]), name = 'Coefficient for s2_shift(t)'))

    #print(list(s1t[0,3750:4250]))
    
    g1.relayout({'xaxis.title': 'Time [ms]'})
    g1.relayout({'xaxis.range': [2500,3500]})
    g1.relayout({'yaxis.showgrid': False })
    g1.relayout({'yaxis.ticks': ''})
    g1.relayout({'yaxis.showticklabels':False})
    g1.relayout({'yaxis.zeroline': False})
    
    g.restyle({'x': [[]], 'y': [[]]})
    g.add_traces(dict(x=tt, y=xt,name = 'Source 1: x(t)'))                     
    g.add_traces(dict(x=tt, y=yt,name = 'Source 2: y(t)'))
    g.relayout({'xaxis.title': 'Time [ms]'})
    g.relayout({'xaxis.range': [2500,3500]})
    
    g2.restyle({'x': [[]], 'y': [[]]})
    
    trace1 = go.Scatter(
        x = [np.real(Cxy_f)],
        y = [np.imag(Cxy_f)],
        name='Coherency_xy(f)'
    )
    
    trace2 = go.Scatter(
        x = [0,np.real(Cxy_f)],
        y = [0,np.imag(Cxy_f)],
        mode = 'lines'
    )

    g2.add_traces(trace1)
    g2.add_traces(trace2)
    g2.relayout({'xaxis.range': [-1.2, 1.2]})
    g2.relayout({'yaxis.range': [-1.2, 1.2]})
    g2.relayout({
    'width':500,
    'height':500,
    'shapes': [
        {
            'type': 'circle',
            'xref': 'x',
            'yref': 'y',
            'x0': -1,
            'y0': -1,
            'x1': 1,
            'y1': 1,
            'opacity': 0.2,
            'fillcolor': 'white',
            'line': {
                'color': 'black',
            }
        }]})
    
    g2.relayout({'title': 'Mean $C_{x,y}(f)$ through all time and all trials'})
    
    trace4 = go.Scatter(
        x = [val for pair in zip(np.zeros(np.shape(cohe)), np.real(cohe)) for val in pair], #y = counts,
        y = [val for pair in zip(np.zeros(np.shape(cohe)), np.imag(cohe)) for val in pair], #x = bins,
        mode = 'lines+markers'
    )
        
    g4.restyle({'x': [[]], 'y': [[]]})
    g4.add_traces(trace4)
    #g4.relayout({'xaxis.title': 'Angle of coherency (0 to 2pi)'})
    g4.relayout({'title': 'Coherency $C_{x,y}(f,t)$ values for every time point'})
    g4.relayout({'xaxis.range': [-1.2, 1.2]})
    g4.relayout({'yaxis.range': [-1.2, 1.2]})
    g4.relayout({
    'width':500,
    'height':500,
    'shapes': [
        {
            'type': 'circle',
            'xref': 'x',
            'yref': 'y',
            'x0': -1,
            'y0': -1,
            'x1': 1,
            'y1': 1,
            'opacity': 0.2,
            'fillcolor': 'white',
            'line': {
                'color': 'black',
            }
        }]})
    
    trace5 = go.Scatter(
        x = np.linspace(0,Fs*time,Fs*time/T),
        y = list(angles),
        name = 'angle',
        mode = 'lines+markers'
    )
    
    trace6 = go.Scatter(
        x = np.linspace(0,Fs*time,Fs*time/T),
        y = list(np.abs(cohe)),
        name = 'amplitude',
        mode = 'lines+markers'
    )
    
    g5.restyle({'x': [[]], 'y': [[]]})
    g5.add_traces(trace5)
    g5.add_traces(trace6)
    #g4.relayout({'xaxis.title': 'Angle of coherency (0 to 2pi)'})
    g5.relayout({'title': 'Coherency $C_{x,y}(f,t)$ values at each time point displayed as phase and amplitude'})
    
g = GraphWidget('https://plot.ly/~chris/4103')
g1 = GraphWidget('https://plot.ly/~chris/4103')
g2 = GraphWidget('https://plot.ly/~chris/4103')
g4 = GraphWidget('https://plot.ly/~chris/4103')
g5 = GraphWidget('https://plot.ly/~chris/4103')

s_all = interactive(regraph,slider_phase=(0,99),slider_noise=(0,100),slider_freq1=(1,80),slider_freq2=(1,80),slider_freq3=(1,80),slider_f=(0,80))
#s_phase.min = 0
#s_phase.max=99
#s_phase.value=3
#s_phase.description = 'Percent of phase shift'


display(s_all)
display(g1)

#display(s_noise)

display(g)

#display(s_f)    

display(g2)
display(g4)
display(g5)

/Users/andrea/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning:

using a non-integer number instead of an integer will result in an error in the future

/Users/andrea/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:14: DeprecationWarning:

using a non-integer number instead of an integer will result in an error in the future

